In [1]:
import numpy as np
import pandas as pd
import openpyxl
import os 
import xlrd
# Setting Directory
os.chdir('C:/Users/TAM/Documents/Vu Quang Nguyen/VACP/')

In [2]:
data_nhapxuatton=pd.read_excel("../VACP/Python NCC Min/Tổng hợp Nhập - Xuất - Tồn YTD.xls",usecols="A:B,G:O",skipfooter=1,header=5)
data_nhapxuatton=data_nhapxuatton.rename(columns={
                                "Unnamed: 0":"ID",
                                "Unnamed: 1":"Tên vật tư",
                                "Số lượng":"Tổng đầu kỳ", 
                                "Giá trị":"Giá trị đầu kỳ",
                                "Số lượng.1":"Tổng nhập", 
                                "Giá trị.1":"Giá trị nhập",
                                "Số lượng.2":"Tổng xuất", 
                                "Giá trị.2":"Giá trị xuất",
                                "Số lượng.3":"Tổng tồn", 
                                "Giá trị.3":"Giá trị tồn" })

 
data_muahang=pd.read_excel("../VACP/Python NCC Min/Sổ chi tiết mua hàng (2).xls",usecols="A,C,F,I:J",skipfooter=1,header=5)
data_muahang=data_muahang.rename(columns={
                                "Unnamed: 0":"Ngày nhập",
                                "Unnamed: 2":"Nhà cung cấp",
                                "Unnamed: 5":"ID",
                                "Số lượng":"Tổng nhập", 
                                "Đơn giá":"Giá nhập"
                                })
# data_nhapxuatton["Giá vốn"]=data_nhapxuatton.loc[data_nhapxuatton["Giá trị xuất"]>0]
data_banhang=pd.read_excel("../VACP/Bao cao Ban hang Power BI/Sổ chi tiết bán hàng.xls",usecols="A,C:D,F,H,K:M",skipfooter=1,header=6)
data_banhang=data_banhang.rename(columns={
                                "Unnamed: 0":"Ngày bán",
                                "Unnamed: 2":"ID Khách hàng",
                                "Unnamed: 3":"Hãng",
                                "Unnamed: 5":"ID",
                                "Unnamed: 7":"Tên hàng",
                                "Số lượng":"Tổng bán", 
                                "Đơn giá gốc":"Giá bán",
                                "Giá bán":"Giá bán giảm"
                                })
data_input=pd.read_excel("../VACP/Python NCC Min/Analysing Price 03 10 - X7/INPUT.xlsx",usecols="B:M")

# data_input.head()
# data_banhang.head()
# data_nhapxuatton.head()

In [ ]:
# Giá vốn
TransCost=0.05
data_nhapxuatton['Giá vốn'] =(1+TransCost)*np.where(data_nhapxuatton["Giá trị xuất"]>0,
                                        data_nhapxuatton["Giá trị xuất"]/data_nhapxuatton["Tổng xuất"],
                                            np.where(data_nhapxuatton["Giá trị tồn"]>0,
                                                data_nhapxuatton["Giá trị tồn"]/data_nhapxuatton["Tổng tồn"],0

                                            )
)

In [3]:
#Mua hàng giá nhập nhỏ nhất
min_mua_hang=data_muahang.groupby(["ID"])["Giá nhập"].min()
data_muahang_min=data_muahang.merge(min_mua_hang,on="ID",how="left",suffixes=('', '_min'))
data_muahang_min=data_muahang_min[data_muahang_min["Giá nhập"]==data_muahang_min["Giá nhập_min"]].drop('Giá nhập_min',axis=1).drop_duplicates(subset="ID",keep='first').reset_index(drop=True)

In [4]:
#Bán hàng gần đây
max_date_banhang=data_banhang.groupby(["ID"])["Ngày bán"].max()
data_banhang_latest=data_banhang.merge(max_date_banhang,on="ID",how="left",suffixes=('', '_max'))
data_banhang_latest=data_banhang_latest[data_banhang_latest["Ngày bán"]==data_banhang_latest["Ngày bán_max"]].drop('Ngày bán_max',axis=1).drop_duplicates(subset="ID",keep='first').reset_index(drop=True)
 

In [7]:
# Gộp dữ liệu Nhập Xuất Tồn + MUA + BÁN
data_nhap_mua_ban=data_nhapxuatton.merge(data_muahang_min,on="ID").merge(data_banhang_latest,on="ID")
data_nhap_mua_ban_filter=data_nhap_mua_ban[["ID","Giá vốn","Ngày nhập","Giá nhập","Nhà cung cấp","Giá bán giảm","Tổng nhập_x","Tổng xuất", "Tổng tồn"]]

In [9]:
data_nhap_mua_ban_filter

,ID,Giá vốn,Ngày nhập,Giá nhập,Nhà cung cấp,Giá bán giảm,Tổng nhập_x,Tổng xuất,Tổng tồn
0,0064662301,6.860570e+06,2022-08-19,6400000.0,NCC-KHANGTHANG,8800000.0,1,1,0
1,016-050-0031HD,1.799820e+06,2020-03-27,1691000.0,HTAUTOHN,2222000.0,1,2,0
2,016-050-0032HD,1.799820e+06,2020-03-27,1691000.0,HTAUTOHN,2222000.0,1,2,0
3,01X49-0056,2.625000e+05,2022-08-23,250000.0,HUNGUUTHINH,300000.0,1,1,0
4,03-R-TT-2166,2.415000e+05,2022-05-04,230000.0,KL-QUANGSAIGON,300000.0,1,1,0
...,...,...,...,...,...,...,...,...,...
2415,ZA-58TKZ3504A3RA,3.354740e+05,2021-06-30,305000.0,NCC-VUGIA,370000.0,407,331,117
2416,ZA-60TKZ3502ARA,3.360000e+05,2022-07-09,320000.0,QUANGVU,360000.0,1,1,0
2417,ZA-62TB0629B28,4.526468e+05,2020-08-25,430000.0,NCC-NAMBAC,490000.0,21,23,0
2418,ZA-68TKB3802BR,4.914000e+05,2021-04-23,468000.0,NCC-NAMBAC,550000.0,0,1,0


In [17]:
data_nhap_mua_ban_filter

,ID,Giá vốn,Ngày nhập,Giá nhập,Nhà cung cấp,Giá bán giảm,Tổng nhập_x,Tổng xuất,Tổng tồn
0,0064662301,6.860570e+06,2022-08-19,6400000.0,NCC-KHANGTHANG,8800000.0,1,1,0
1,016-050-0031HD,1.799820e+06,2020-03-27,1691000.0,HTAUTOHN,2222000.0,1,2,0
2,016-050-0032HD,1.799820e+06,2020-03-27,1691000.0,HTAUTOHN,2222000.0,1,2,0
3,01X49-0056,2.625000e+05,2022-08-23,250000.0,HUNGUUTHINH,300000.0,1,1,0
4,03-R-TT-2166,2.415000e+05,2022-05-04,230000.0,KL-QUANGSAIGON,300000.0,1,1,0
...,...,...,...,...,...,...,...,...,...
2415,ZA-58TKZ3504A3RA,3.354740e+05,2021-06-30,305000.0,NCC-VUGIA,370000.0,407,331,117
2416,ZA-60TKZ3502ARA,3.360000e+05,2022-07-09,320000.0,QUANGVU,360000.0,1,1,0
2417,ZA-62TB0629B28,4.526468e+05,2020-08-25,430000.0,NCC-NAMBAC,490000.0,21,23,0
2418,ZA-68TKB3802BR,4.914000e+05,2021-04-23,468000.0,NCC-NAMBAC,550000.0,0,1,0


In [19]:
data_report=data_input.merge(data_nhap_mua_ban_filter, on="ID",how="left").drop_duplicates(subset=['ID','Nhà sản xuất']).reset_index(drop=True)
data_report

,ID,ID Item,Description,PCS/PACK,Exists Stocks,Qty request,Unit price,Mã nguyên tệ,Nhà sản xuất,Số lượng cần nhập,Application,Hãng,Giá vốn,Ngày nhập,Giá nhập,Nhà cung cấp,Giá bán giảm,Tổng nhập_x,Tổng xuất,Tổng tồn
0,11189-62R10-000,"XL7, ERTIGA","GASKET,CYLINDER HEAD COVER",NaN,NaN,NaN,2.51,USD,NK-IDAS,NaN,X7,SUZUKI,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
1,11141-73R00-000,"XL7, ERTIGA","GASKET,CYL HEAD",NaN,NaN,NaN,6.63,USD,NK-IDAS,NaN,X7,SUZUKI,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2,11170-73R00-000,"XL7, ERTIGA","COVER ASSY,CYLINDER HEAD",NaN,NaN,NaN,26.17,USD,NK-IDAS,NaN,X7,SUZUKI,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
3,18640-73R00-000,"XL7, ERTIGA","SENSOR ASSY,KNOCK",NaN,NaN,NaN,3.78,USD,NK-IDAS,NaN,X7,SUZUKI,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
4,16510-61J00-000,"XL7, ERTIGA",OIL FILTER,NaN,NaN,NaN,1.98,USD,NK-IDAS,NaN,X7,SUZUKI,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
658,48068-F4020,NaN,NaN,NaN,NaN,NaN,190.22,USD,NK-TRISTAN,NaN,CROSS,TOYOTA,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
659,48069-F4020,NaN,NaN,NaN,NaN,NaN,200.73,USD,NK-TRISTAN,NaN,CROSS,TOYOTA,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
660,43330-09A40,NaN,NaN,NaN,NaN,NaN,45.36,USD,NK-TRISTAN,NaN,CROSS,TOYOTA,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
661,81105-0A080,NaN,NaN,NaN,NaN,NaN,NaN,USD,NK-TRISTAN,NaN,CROSS,TOYOTA,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# EXPORT FILE DATA
with pd.ExcelWriter('../VACP/Python NCC Min/Analysing Price 03 10 - X7/Export_data.xlsx') as writer:
    data_nhap_mua_ban_filter.to_excel(writer, sheet_name='Nhập Mua Bán')
    data_report.to_excel(writer, sheet_name='Report')

